In [1]:
import pandas as pd
import numpy as np

# Load the data
full_data = pd.read_csv('/home/data/catechol_full_data_yields.csv')
single_data = pd.read_csv('/home/data/catechol_single_solvent_yields.csv')

print("Full data shape:", full_data.shape)
print("Single solvent data shape:", single_data.shape)
print("\nFull data columns:")
print(full_data.columns.tolist())
print("\nSingle data columns:")
print(single_data.columns.tolist())

Full data shape: (1227, 19)
Single solvent data shape: (656, 13)

Full data columns:
['EXP NUM', 'SOLVENT A NAME', 'SOLVENT B NAME', 'SolventB%', 'Residence Time', 'Temperature', 'SM', 'Product 2', 'Product 3', 'SM SMILES', 'Product 2 SMILES', 'Product 3 SMILES', 'SOLVENT A SMILES', 'SOLVENT B SMILES', 'SOLVENT A Ratio', 'SOLVENT B Ratio', 'Reaction SMILES A', 'Reaction SMILES B', 'RAMP NUM']

Single data columns:
['EXP NUM', 'Residence Time', 'Temperature', 'SM', 'Product 2', 'Product 3', 'SM SMILES', 'Product 2 SMILES', 'Product 3 SMILES', 'SOLVENT NAME', 'SOLVENT SMILES', 'SOLVENT Ratio', 'Reaction SMILES']


In [2]:
# Explore targets and features
print("Target statistics (Single solvent):")
print(single_data[['SM', 'Product 2', 'Product 3']].describe())
print("\nTarget statistics (Full data):")
print(full_data[['SM', 'Product 2', 'Product 3']].describe())

print("\n\nUnique solvents in single data:", single_data['SOLVENT NAME'].nunique())
print("Unique solvent pairs in full data:", full_data[['SOLVENT A NAME', 'SOLVENT B NAME']].drop_duplicates().shape[0])

print("\nSolvent names in single data:")
print(single_data['SOLVENT NAME'].unique())

Target statistics (Single solvent):
               SM   Product 2   Product 3
count  656.000000  656.000000  656.000000
mean     0.522192    0.149932    0.123380
std      0.360229    0.143136    0.131528
min      0.000000    0.000000    0.000000
25%      0.145001    0.012976    0.009445
50%      0.656558    0.102813    0.078298
75%      0.857019    0.281654    0.193353
max      1.000000    0.463632    0.533768

Target statistics (Full data):
                SM    Product 2    Product 3
count  1227.000000  1227.000000  1227.000000
mean      0.495178     0.164626     0.143668
std       0.379425     0.153467     0.145779
min       0.000000     0.000000     0.000000
25%       0.068573     0.012723     0.012260
50%       0.606454     0.117330     0.094413
75%       0.877448     0.308649     0.254630
max       1.083254     0.463632     0.533768


Unique solvents in single data: 24
Unique solvent pairs in full data: 13

Solvent names in single data:
['Methanol' 'Ethylene Glycol [1,2-Ethanedio

In [4]:
# Check the lookup tables for feature dimensions
spange = pd.read_csv('/home/data/spange_descriptors_lookup.csv', index_col=0)
acs_pca = pd.read_csv('/home/data/acs_pca_descriptors_lookup.csv', index_col=0)
drfps = pd.read_csv('/home/data/drfps_catechol_lookup.csv', index_col=0)
fragprints = pd.read_csv('/home/data/fragprints_lookup.csv', index_col=0)

print("Spange descriptors shape:", spange.shape)
print("ACS PCA descriptors shape:", acs_pca.shape)
print("DRFP shape:", drfps.shape)
print("Fragprints shape:", fragprints.shape)

print("\nSpange columns:", spange.columns.tolist())

Spange descriptors shape: (26, 13)
ACS PCA descriptors shape: (24, 5)
DRFP shape: (24, 2048)
Fragprints shape: (24, 2133)

Spange columns: ['dielectric constant', 'ET(30)', 'alpha', 'beta', 'pi*', 'SA', 'SB', 'SP', 'SdP', 'N', 'n', 'f(n)', 'delta']


In [5]:
# Check CV structure - how many folds for each task
print("Single solvent CV: Leave-one-solvent-out")
print(f"  - 24 solvents = 24 folds")
print(f"  - Each fold tests on ~{656/24:.0f} samples")

print("\nFull data CV: Leave-one-ramp-out")
print(f"  - 13 solvent pairs = 13 folds")
print(f"  - Each fold tests on ~{1227/13:.0f} samples")

# Check the solvent pairs
print("\nSolvent pairs in full data:")
pairs = full_data[['SOLVENT A NAME', 'SOLVENT B NAME']].drop_duplicates()
for _, row in pairs.iterrows():
    count = ((full_data['SOLVENT A NAME'] == row['SOLVENT A NAME']) & 
             (full_data['SOLVENT B NAME'] == row['SOLVENT B NAME'])).sum()
    print(f"  {row['SOLVENT A NAME']} - {row['SOLVENT B NAME']}: {count} samples")

Single solvent CV: Leave-one-solvent-out
  - 24 solvents = 24 folds
  - Each fold tests on ~27 samples

Full data CV: Leave-one-ramp-out
  - 13 solvent pairs = 13 folds
  - Each fold tests on ~94 samples

Solvent pairs in full data:
  Methanol - Ethylene Glycol [1,2-Ethanediol]: 122 samples
  1,1,1,3,3,3-Hexafluoropropan-2-ol - 2-Methyltetrahydrofuran [2-MeTHF]: 124 samples
  Cyclohexane - IPA [Propan-2-ol]: 104 samples
  Water.Acetonitrile - Acetonitrile: 125 samples
  Acetonitrile - Acetonitrile.Acetic Acid: 125 samples
  2-Methyltetrahydrofuran [2-MeTHF] - Diethyl Ether [Ether]: 124 samples
  2,2,2-Trifluoroethanol - Water.2,2,2-Trifluoroethanol: 125 samples
  DMA [N,N-Dimethylacetamide] - Decanol: 110 samples
  Ethanol - THF [Tetrahydrofuran]: 127 samples
  Dihydrolevoglucosenone (Cyrene) - Ethyl Acetate: 36 samples
  MTBE [tert-Butylmethylether] - Butanone [MEK]: 34 samples
  tert-Butanol [2-Methylpropan-2-ol] - Dimethyl Carbonate: 36 samples
  Methyl Propionate - Ethyl Lactate: 3

In [6]:
# Check the numeric features range
print("Numeric features (single solvent):")
print(single_data[['Residence Time', 'Temperature']].describe())

print("\nNumeric features (full data):")
print(full_data[['Residence Time', 'Temperature', 'SolventB%']].describe())

# Check if targets sum to 1 (mass balance)
print("\nTarget sum statistics (single):")
print((single_data['SM'] + single_data['Product 2'] + single_data['Product 3']).describe())

print("\nTarget sum statistics (full):")
print((full_data['SM'] + full_data['Product 2'] + full_data['Product 3']).describe())

Numeric features (single solvent):
       Residence Time  Temperature
count      656.000000   656.000000
mean        12.621262   201.181480
std          4.184125    21.349418
min          2.001019   175.000000
25%         11.269690   175.000000
50%         15.017208   200.000000
75%         15.017208   225.000000
max         15.017208   225.000000

Numeric features (full data):
       Residence Time  Temperature    SolventB%
count     1227.000000  1227.000000  1227.000000
mean        13.726828   200.937287     0.435865
std          3.280033    23.118608     0.400336
min          2.001019   175.000000     0.000000
25%         14.988550   175.000000     0.000000
50%         15.000393   200.000000     0.330153
75%         15.017208   225.000000     0.858481
max         15.017208   225.000000     1.000000

Target sum statistics (single):
count    656.000000
mean       0.795504
std        0.194306
min        0.028752
25%        0.708417
50%        0.849648
75%        0.927955
max        1.0